In [0]:
dbutils.fs.ls("s3://goldyyy-de-practice/Assignment_Files/Customer.xlsx")

Out[79]: [FileInfo(path='s3://goldyyy-de-practice/Assignment_Files/Customer.xlsx', name='Customer.xlsx', size=89424, modificationTime=1727727077000)]

In [0]:
%run "./Assignment - utilities"

In [0]:
    from pyspark.sql import SparkSession
    from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
    import pyspark.sql.functions as F

In [0]:
spark = SparkSession.builder \
        .appName("createProfitReport") \
        .getOrCreate()

### Ingest Product File

In [0]:
### Create Product Schema
productSchema = StructType([
    StructField("Product ID", StringType(), True),
    StructField("Category", StringType(), True),
    StructField("Sub-Category", StringType(), True),
    StructField("Product Name", StringType(), True),
    StructField("State", StringType(), True),
    StructField("Price per product", DoubleType(), True),
])

### Read Product File from Data Lake
product_df = spark.read.format('csv')\
    .option("header", "true") \
    .schema(productSchema) \
    .option("quote", "\"") \
    .option("escape", "\"") \
    .load("s3://goldyyy-de-practice/Assignment_Files/Product.csv")

### Update Column Names
productColumnMapping = {"Product ID": "product_id",
                 "Category": "category",
                 "Sub-Category": "sub_category",
                 "Product Name": "product_name",
                 "State": "state",
                 "Price Per Product": "price_per_product"}

product_renamed_df = renameColumns(product_df, productColumnMapping)


### Add audit columns
columnsWithValues = {
    "ingestion_date": F.current_timestamp()
}

product_final_df = addColumns(product_renamed_df, columnsWithValues)


### Save Product Data in Delta table
filePath = "dbfs:/user/raw/raw_product"
schema = "raw"
tableName = "raw_product"
saveDataInTable(product_final_df, filePath, schema, tableName)


### Ingest Customer File

In [0]:
### Create Customer Schema
customerSchema = StructType([
    StructField("Customer ID", StringType(), True),
    StructField("Customer Name", StringType(), True),
    StructField("email", StringType(), True),
    StructField("phone", StringType(), True),
    StructField("address", StringType(), True),
    StructField("Segment", StringType(), True),
    StructField("Country", StringType(), True),
    StructField("City", StringType(), True),
    StructField("State", StringType(), True),
    StructField("Postal Code", DoubleType(), True),
    StructField("Region", StringType(), True),
])

### Read Customer File from Data Lake
customer_df = spark.read.format('com.crealytics.spark.excel') \
    .option("header", "true") \
    .schema(customerSchema) \
    .option("quote", "\"") \
    .option("escape", "\"") \
    .load("s3://goldyyy-de-practice/Assignment_Files/Customer.xlsx")

### Update Column Names
customerColumnMapping = {"Customer ID": "customer_id",
                 "Customer Name": "customer_name",
                 "Segment": "segment",
                 "Country": "country",
                 "City": "city",
                 "State": "state",
                 "Postal Code": "postal_code",
                 "Region": "region"
}

customer_renamed_df = renameColumns(customer_df, customerColumnMapping)


### Add audit columns
columnsWithValues = {
    "ingestion_date": F.current_timestamp()
}

customer_final_df = addColumns(customer_renamed_df, columnsWithValues)


### Save Customer Data in Delta table
filePath = "dbfs:/user/raw/raw_customer"
schema = "raw"
tableName = "raw_customer"
saveDataInTable(customer_final_df, filePath, schema, tableName)

### Ingest Orders File

In [0]:
### Create Orders schema
orderSchema = StructType([
    StructField("Row ID", IntegerType(), True),
    StructField("Order ID", StringType(), True),
    StructField("Order Date", StringType(), True),
    StructField("Ship Date", StringType(), True),
    StructField("Ship Mode", StringType(), True),
    StructField("Customer ID", StringType(), True),
    StructField("Product ID", StringType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("Price", StringType(), True),
    StructField("Discount", DoubleType(), True),
    StructField("Profit", DoubleType(), True)
])

### Read Orders file from data lake
orders_df = spark.read.option("multiline", "true").schema(orderSchema).json("s3://goldyyy-de-practice/Assignment_Files/Order.json")

### Data pre-processing

orders_formatted_df = orders_df.withColumn("Cleaned Price", F.regexp_replace(F.col("Price"), "%", "").cast(DoubleType())).withColumn("order_date", F.to_date("Order Date", 'd/m/yyyy')).withColumn("ship_date", F.to_date("Ship Date", 'd/m/yyyy'))

### Drop columns that are not required
columnsToDrop = ["Ship Date", "Order Date", "Price"]
orders_cleaned_df = dropColumns(orders_formatted_df, columnsToDrop)

### Update Column Names
ordersColumnMapping = {
    "Row ID": "row_id",
    "Order ID": "order_id",
    "Ship Mode": "ship_mode",
    "Customer ID": "customer_id",
    "Product ID": "product_id",
    "Quantity": "quantity",
    "Cleaned Price": "price",
    "Discount": "discount",
    "Profit": "profit"
}

orders_renamed_df = renameColumns(orders_cleaned_df, ordersColumnMapping)


### Add audit columns
columnsWithValues = {
    "ingestion_date": F.current_timestamp()
}

orders_final_df = addColumns(orders_renamed_df, columnsWithValues)


### Save Customer Data in Delta table
filePath = "dbfs:/user/raw/raw_orders"
schema = "raw"
tableName = "raw_orders"
saveDataInTable(orders_final_df, filePath, schema, tableName)

